# Core Component: Message Bus

The `MessageBus` is the central communication backbone for the multi-agent system. It implements the **publish-subscribe** design pattern, which allows agents to communicate without being directly aware of each other. This decoupling is a critical feature for building complex, scalable, and maintainable agent-based systems.

### How it Works
1. **Topics**: Messages are organized by topics, which are simple strings (e.g., `"state.reservoir.level"`).
2. **Publishing**: An agent can send a `message` (a Python dictionary) to a specific `topic`.
3. **Subscribing**: An agent can register a `listener` function (a callback) to a specific `topic`. 

When a message is published to a topic, the message bus immediately invokes all listener functions that are subscribed to that exact topic, passing the message to them.

**Note:** This is a simple implementation and does not support advanced features like wildcard subscriptions.

## Demonstration

In the example below, we will create a message bus and several listeners. We will subscribe them to different topics and then publish a series of messages. The output will show which listener received which message, demonstrating the topic-based routing.

In [ ]:
from swp.central_coordination.collaboration.message_bus import MessageBus, Message

# 1. Create the Message Bus
bus = MessageBus()

# 2. Define some listener functions
def reservoir_level_listener(message: Message):
    print(f"[RESERVOIR LISTENER] Received: {message}")

def gate_listener(message: Message):
    print(f"[GATE LISTENER] Received: {message}")
    
def all_commands_listener(message: Message):
    print(f"[COMMANDS LISTENER] Received: {message}")

# 3. Subscribe listeners to topics
print("--- Subscribing Listeners ---")
bus.subscribe("state.reservoir.level", reservoir_level_listener)
bus.subscribe("state.gate.opening", gate_listener)
bus.subscribe("command.gate.setpoint", all_commands_listener)
bus.subscribe("command.pump.status", all_commands_listener)
print("----------------------------\n")

# 4. Publish messages
print("--- Publishing Messages ---")
print("Publishing a reservoir level...")
bus.publish("state.reservoir.level", {'value': 15.2, 'units': 'm'})

print("\nPublishing a gate command...")
bus.publish("command.gate.setpoint", {'value': 0.8, 'units': 'percent'})

print("\nPublishing a gate state (no active listener)...")
bus.publish("state.gate.opening", {'value': 0.75})

print("\nPublishing a pump command...")
bus.publish("command.pump.status", {'value': 1, 'note': 'turn on'})
print("---------------------------")

### Analysis of Output

As the output shows:
- The `reservoir_level_listener` only received the message published to `state.reservoir.level`.
- The `gate_listener` only received the message for `state.gate.opening`.
- The `all_commands_listener` received messages published to both `command.gate.setpoint` and `command.pump.status`, because we subscribed it to both topics individually.
- No listener was invoked for the `state.gate.opening` message because the `gate_listener` was subscribed *after* the message was published. This demonstrates that subscriptions are not retroactive.